In [1]:
from siamese_dataloader import Siamese_dataset, Siamese_dataloader
from siamese_network import SiameseNetwork, SiameseNetworkToy, SiameseNetworkToyConv
from torch import nn as nn
import torch
from torch import device, cuda, no_grad, cat
import torch.optim as optim
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt
from torch.nn import functional as F

In [2]:
device = 'cuda:0' if cuda.is_available() else 'cpu'
print(f"Using device: {device}")

SEED = 2024

root="../train_xml"
# Raw data
subset = np.logical_not(np.load("data/missing_file_names_mask.npy")) # Makes losses adapt to the selected mouse_records
losses = np.load("data/resnet50_losses_final_weights.npy")[subset]
mouse_records = np.load("data/mouse_record_interpolated.npy")
indices_broken = np.load("data/mouse_records_broken.npy")


# Clean data aka reasonable estimated time
estimate_times=np.load('data/sample_estimate_times.npy')
clean_subset = estimate_times[subset] < 3000
clean_subset = np.logical_and(clean_subset, estimate_times[subset] > 0)
clean_subset = np.logical_and(clean_subset, np.logical_not(indices_broken))

losses = losses[clean_subset]
mouse_records = mouse_records[clean_subset]#[:,:5,:]

# Train and test split
assert len(losses) == len(mouse_records)




Using device: cuda:0


In [11]:
indices = list(np.arange(len(losses)))
indices_randomized = np.random.choice(indices, int(len(indices)), replace=False)
train_indices = indices_randomized[:int(0.75*len(indices))]
test_indices = indices_randomized[int(0.8*len(indices)):]
val_indices = indices_randomized[int(0.75*len(indices)):int(0.8*len(indices))]

BATCH_SIZE = 4
mouse_record_shape= (30,2)

train_indices = train_indices[:4] # Lets overfit
losses_train = losses[train_indices]
mouse_records_train = mouse_records[train_indices]
dataset_train = Siamese_dataset(losses_train, mouse_records_train, seed=SEED)
dataloader_train = Siamese_dataloader(dataset_train, batch_size=BATCH_SIZE, num_workers=8, shuffle=True, mouse_record_shape=mouse_record_shape).run()

test_indices=test_indices[:1000]
losses_test = losses[test_indices]
mouse_records_test = mouse_records[test_indices]
dataset_test = Siamese_dataset(losses_test, mouse_records_test, seed=SEED)
dataloader_test = Siamese_dataloader(dataset_test, batch_size=BATCH_SIZE, num_workers=8, shuffle=True, mouse_record_shape=mouse_record_shape).run()

val_indices=val_indices[:256]
losses_val = losses[val_indices]
mouse_records_val = mouse_records[val_indices]
dataset_val = Siamese_dataset(losses_val, mouse_records_val, seed=SEED)
dataloader_val = Siamese_dataloader(dataset_val, batch_size=BATCH_SIZE, num_workers=8, shuffle=True, mouse_record_shape=mouse_record_shape).run()

len(dataset_train), len(dataset_test), len(dataset_val)

(2, 500, 128)

In [22]:
all_weights = []
for _, ( inputs_0, inputs_1, labels, weights) in enumerate(dataloader_train, 0):
    inputs_0, inputs_1, labels, weights = inputs_0.to(device), inputs_1.to(device), labels.to(device), weights.to(device)
    all_weights.append(weights)
all_weights = torch.cat(all_weights)
all_weights.sum(), inputs_0[0][:3], inputs_1[0][:3], inputs_0[1][:3], inputs_1[1][:3], all_weights, labels

(tensor(1., device='cuda:0'),
 tensor([[0.3456, 0.0029],
         [0.4778, 0.3784],
         [0.4702, 0.5510]], device='cuda:0'),
 tensor([[0.6852, 0.0377],
         [0.7407, 0.6075],
         [0.0000, 0.0000]], device='cuda:0'),
 tensor([[0.8889, 0.0545],
         [0.8245, 0.1996],
         [0.8836, 0.0800]], device='cuda:0'),
 tensor([[0.7593, 0.9772],
         [0.7066, 0.8459],
         [0.6759, 0.7534]], device='cuda:0'),
 tensor([[0.],
         [1.]], device='cuda:0'),
 tensor([[0.],
         [1.]], device='cuda:0'))

In [21]:
inputs_0[0][:3], inputs_1[0][:3], '\n',inputs_0[1][:3], inputs_1[1][:3], inputs_0[2][:3], inputs_1[2][:3], inputs_0[3][:3], inputs_1[3][:3], all_weights, labels


(tensor([[0.4383, 0.0573],
         [0.4748, 0.3499],
         [0.4822, 0.3672]], device='cuda:0'),
 tensor([[0.7193, 0.9922],
         [0.7018, 0.9062],
         [0.7018, 0.9062]], device='cuda:0'),
 '\n',
 tensor([[0.8960, 0.6480],
         [0.8247, 0.6480],
         [0.7214, 0.6446]], device='cuda:0'),
 tensor([[0.8580, 0.9561],
         [0.2331, 0.4893],
         [0.1950, 0.4354]], device='cuda:0'),
 tensor([[0.7193, 0.9922],
         [0.7018, 0.9062],
         [0.7018, 0.9062]], device='cuda:0'),
 tensor([[0.4383, 0.0573],
         [0.4748, 0.3499],
         [0.4822, 0.3672]], device='cuda:0'),
 tensor([[0.8580, 0.9561],
         [0.2331, 0.4893],
         [0.1950, 0.4354]], device='cuda:0'),
 tensor([[0.8960, 0.6480],
         [0.8247, 0.6480],
         [0.7214, 0.6446]], device='cuda:0'),
 tensor([[1.],
         [0.],
         [1.],
         [0.]], device='cuda:0'),
 tensor([[1.],
         [0.],
         [0.],
         [0.]], device='cuda:0'))

In [9]:
labels

tensor([[1.],
        [0.]], device='cuda:0')

In [9]:
inputs_0[0][:3], inputs_1[0][:3], inputs_0[-1][:3], inputs_1[-1][:3], labels

(tensor([[0.1728, 0.0087],
         [0.2161, 0.0462],
         [0.2348, 0.0529]], device='cuda:0'),
 tensor([[0.7932, 0.0139],
         [0.0000, 0.0000],
         [0.5159, 0.9860]], device='cuda:0'),
 tensor([[0.7315, 1.0000],
         [0.6862, 0.8160],
         [0.6473, 0.6975]], device='cuda:0'),
 tensor([[0.4474, 0.0206],
         [0.5335, 0.2290],
         [0.5653, 0.3286]], device='cuda:0'),
 tensor([[0.],
         [1.]], device='cuda:0'))

In [10]:
i = np.random.randint(0,10000)
i, mouse_records[i]

(8216,
 array([[0.9444444 , 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
        [0.91049385, 0.6064815 ],
       

In [79]:
def train_one_epoch(train_loader, net, criterion, optimizer, epoch):
    running_loss = 0.0
    predictions = []
    all_labels = []
    for _, (inputs_0, inputs_1, labels, weights) in enumerate(train_loader, 0):
        inputs_0, inputs_1, labels, weights = inputs_0.to(device), inputs_1.to(device), labels.to(device), weights.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs_0, inputs_1)
        loss = criterion(outputs, labels, weight=weights, size_average=False)
        loss.backward()
        optimizer.step()
        weights = weights.flatten()

        predictions.append(outputs[weights==1])
        all_labels.append(labels[weights==1])
        running_loss += loss.item()
    preds = cat(predictions)>0.5
    accuracy = (preds == cat(all_labels)).sum().item() / len(cat(predictions))
    return accuracy, running_loss

def test_one_epoch(loader, net, criterion):
    with no_grad():
        loss = 0
        predictions = []
        all_labels = []
        for _, (inputs_0, inputs_1, labels, weights) in enumerate(loader, 0):
            inputs_0, inputs_1, labels = inputs_0.to(device), inputs_1.to(device), labels.to(device)
            weights = weights.flatten()
            outputs = net(inputs_0, inputs_1)
            predictions.append(outputs[weights==1])
            all_labels.append(labels[weights==1])
            loss += criterion(outputs, labels).item()

        predictions = cat(predictions)
        all_labels = cat(all_labels)
    preds = predictions > 0.5
    accuracy = (preds == all_labels).sum().item() / len(predictions)
    return accuracy, loss

In [82]:
net = SiameseNetworkToyConv()
print(f"The number of parameters is {sum(p.numel() for p in net.parameters())}")
net.to(device)

criterion = F.binary_cross_entropy
# criterion = F.hinge_embedding_loss
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.8)

accuracy, loss = test_one_epoch(dataloader_test, net, criterion)
print(f"Initial test accuracy is {accuracy} and loss is {loss}")

The number of parameters is 103445
Initial test accuracy is 0.46551724137931033 and loss is 93.52345123887062


In [33]:
inputs_0,inputs_1,labels,weights = next(iter(dataloader_train))

In [15]:
all_weights = []
for _, ( inputs_0, inputs_1, labels, weights) in enumerate(dataloader_train, 0):
    inputs_0, inputs_1, labels, weights = inputs_0.to(device), inputs_1.to(device), labels.to(device), weights.to(device)
    all_weights.append(weights)
all_weights = torch.cat(all_weights)
all_weights.sum()

tensor(1., device='cuda:0')

In [59]:
len(dataset_train)

2

In [86]:
inputs_0,inputs_1,labels,weights = next(iter(dataloader_train))
inputs_0, inputs_1, labels, weights = inputs_0.to(device), inputs_1.to(device), labels.to(device), weights.to(device)
# zero the parameter gradients
optimizer.zero_grad()

# forward + backward + optimize
outputs = net(inputs_0, inputs_1)
preds = outputs > 0.5
loss = criterion(outputs, labels, weight=weights)
loss.backward()
optimizer.step()
outputs, loss, labels, preds, weights,( preds[weights==1]==labels[weights==1]).sum()/weights.sum(),  #inputs_0, inputs_1

(tensor([[0.4957],
         [0.4941]], device='cuda:0', grad_fn=<SigmoidBackward0>),
 tensor(0.3508, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>),
 tensor([[1.],
         [0.]], device='cuda:0'),
 tensor([[False],
         [False]], device='cuda:0'),
 tensor([[1.],
         [0.]], device='cuda:0'),
 tensor(0., device='cuda:0'))

In [87]:
EPOCHS = 100
training_accuracies = []
training_losses = []
val_accuracies = []
val_losses = []
for _, epoch in enumerate(tqdm(range(EPOCHS))):
    train_accuracy, loss = train_one_epoch(dataloader_train, net, criterion, optimizer, epoch)
    val_accuracy, v_loss = test_one_epoch(dataloader_val, net, criterion)
    training_accuracies.append(train_accuracy)
    training_losses.append(loss)
    val_accuracies.append(val_accuracy)
    val_losses.append(v_loss)

    print(f"Epoch: {epoch}/{EPOCHS} | Loss : {loss} | Train accuracy: {train_accuracy} | Val accuracy: {val_accuracy} | Val loss: {v_loss}")

accuracy, loss = test_one_epoch(dataloader_test, net, criterion)
print(f"Finished Training, final test accuracy is {accuracy} and loss is {loss}, final training accuracy is {train_accuracy}")

  1%|          | 1/100 [00:00<00:56,  1.74it/s]

Epoch: 0/100 | Loss : 0.29695820808410645 | Train accuracy: 1.0 | Val accuracy: 0.5416666666666666 | Val loss: 23.69816419482231


  2%|▏         | 2/100 [00:00<00:44,  2.20it/s]

Epoch: 1/100 | Loss : 0.7588624954223633 | Train accuracy: 0.0 | Val accuracy: 0.5416666666666666 | Val loss: 24.010351479053497


  3%|▎         | 3/100 [00:01<00:41,  2.36it/s]

...

 99%|█████████▉| 99/100 [00:38<00:00,  2.60it/s]

Epoch: 98/100 | Loss : 0.7054397463798523 | Train accuracy: 0.0 | Val accuracy: 0.4791666666666667 | Val loss: 22.847467362880707


100%|██████████| 100/100 [00:38<00:00,  2.58it/s]

Epoch: 99/100 | Loss : 0.2383473664522171 | Train accuracy: 1.0 | Val accuracy: 0.6041666666666666 | Val loss: 25.218609541654587


Finished Training, final test accuracy is 0.5172413793103449 and loss is 95.3354980647564, final training accuracy is 1.0
